# PlayHQ Fixture Scraping

This system allows to scrape games from [PlayHQ](http://playhq.com/
) via its Public [API](https://support.playhq.com/hc/en-au/sections/4405422358297-PlayHQ-APIs) and produce CSV files ready to be uploaded to [TeamApp](https://brunswickmagicbasketball.teamapp.com/). (Private APIs are not available to clubs and associations.)

The *Public* APIs only require a header parameters to get a successful response, which includes `x-api-key` (also referred to as the Client ID) and `x-phq-tenant` (refers to the sport/association - in this case `bv`).

Detailed reference documentation for PlayHQ API can be found [here](https://docs.playhq.com/tech).

Contact Sebastian Sardina  (sssardina@gmail.com)

In [1]:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

import json
import pandas as pd
import re
import csv
import os
import calendar, datetime, pytz


import playhq_api as phq

# import dtale

## 1. Set-up application

We start by creating a connection to the PlayHQ Public API.

To do so, we need to define the ids and keys for the club to access the PlayHQ API.

The organization Play-HQ ID can be obtained from the admin PlayHQ URL:

```
https://bv.playhq.com/org/<ORG_ID>/competitions
```

The *Public* APIs only require the below header parameters to get a successful response:

- `x-api-key` (also referred to as the Client ID) will be provided by PlayHQ when you request access to the public API via their [support page](https://support.playhq.com/hc/en-au) or email support@playhqsupport.zendesk.com.
- `x-phq-tenant` usually refers to the sport/association - in this case '`bv`'.

The feature to create new API credentials is disabled for you and can only be actioned by a Super Administrator role within the Play HQ portal. Please use the credentials provided to call the PlayHQ public APIs.

In [2]:
CLUB_NAME = "Brunswick Magic Basketball Club"
X_TENANT="bv"
ORG_ID="8c4d5431-eaa5-4644-82ac-992abe224b88"
GAME_DATE = phq.next_day(calendar.SATURDAY) # get the date of the upcoming Saturday (game day in competition)
TIMEZONE = 'Australia/Melbourne'

# Provide the x-api-key either explicitly or via a file
X_API_KEY="<YOUR PLAYHQ ADMIN KEY>"

if os.path.exists('x_api_key.txt'):
    with open('x_api_key.txt') as f:
        X_API_KEY = f.readlines()[0]

phq_club = phq.PlayHQ(CLUB_NAME, ORG_ID, X_API_KEY, X_TENANT, TIMEZONE)

print(f"Set-up games for club {CLUB_NAME} for upcoming Saturday is: {GAME_DATE.strftime('%A %B %d of %Y')}")

Set-up games for club Brunswick Magic Basketball Club for upcoming Saturday is: Saturday August 06 of 2022


Next define the data for the season that we are interested in.

In [3]:
SEASON = "Winter 2022"

DESC_BYE_TAPP = "Sorry, no game for the team in this round."
DESC_TAPP = """RSVP is YES by default - if you cannot make it, please let your Team Manager know as soon as possible.
Opponent: {opponent}
Venue: {venue} ({court})
Address: {address} {address_tips}
Google Maps coord: https://maps.google.com/?q={coord}

- Please ensure you arrive early and ready.
- Remember that shorts should have no pockets, players should not wear bracelets/watch as it is a risk of injury.
- 45 min schedule with 18 min halves.
- Each team needs to provide a scorer. TMs, please consider a roster.
- Players should not bring balls into the venue - game balls provided by Magic in coach's equipment bag.
- Beginners refs will be wearing green shirts. Please support and respect them through a POSITIVE sideline behaviour.

Check the game in PlayHQ: {url_game}
Check the round in PlayHQ: {url_grade}
All clubs in PlayHQ: https://bit.ly/bmbc-w22
"""

PATH_SAVE_CSV='Brunswick_Magics/2022.01.Winter/fixture'

## 2. Get the teams and upcoming games for those teams

First, get the competition id for the season the organisation is in.

In [4]:
season_id = phq_club.get_season_id(SEASON)
teams_df = phq_club.get_season_teams(season_id)
teams_df

2022-08-04 12:19:09 DEBUG Seasons id for season *Winter 2022*: 80d8cc4b-bba6-4e2d-92da-053be5107336
2022-08-04 12:19:09 DEBUG Competition id for season *Winter 2022*: 81fe8d81-d087-4a63-b4fb-87bc85e9d8ac


,id,name,grade.id,grade.name,grade.url,age
0,d7486008-b2fe-47db-8011-01265eaf1cfe,Magic U8 Mixed Purple,be7076cb-a1de-4033-8082-b52c8c149861,Saturday U8 Mixed Division 1/2,https://www.playhq.com/basketball-victoria/org...,8
1,8d635867-0718-4f70-a1b8-1a224992d294,Magic U16 Girls Gold,19b86372-2ad5-4a5a-9592-467f826f7d63,Saturday U16 Girls Division 1/2,https://www.playhq.com/basketball-victoria/org...,16
2,69d69214-641d-4ece-a2ce-ca9534865553,Magic U16 Boys Purple,be4b2b9c-0bc1-44e2-ac3a-b89bb10230f8,Saturday U16 Boys Division 3,https://www.playhq.com/basketball-victoria/org...,16
3,135c3bab-3004-4426-b9c6-3669b48ce271,Magic U16 Boys Gold,ceac1ec4-c86b-47c6-9e67-124d923ea713,Saturday U16 Boys Division 2,https://www.playhq.com/basketball-victoria/org...,16
4,99f369ed-5c63-40e7-8591-8cd18fba7ec0,Magic U14 Boys Gold,d900616b-0f52-4d3e-8090-4eaf877491bb,Saturday U14 Boys Division 5,https://www.playhq.com/basketball-victoria/org...,14
5,8b2bc68b-9fda-435c-9c13-297d1d3187d1,Magic U14 Girls Purple,b0785b1f-bec6-448b-a76d-83e1a6371fb3,Saturday U14 Girls Division 3,https://www.playhq.com/basketball-victoria/org...,14
6,9ebeb141-4023-4744-84f1-f58957b966b2,Magic U14 Girls Gold,b0785b1f-bec6-448b-a76d-83e1a6371fb3,Saturday U14 Girls Division 3,https://www.playhq.com/basketball-victoria/org...,14
7,2b844e90-6575-47bc-bb49-f55840f8e6f8,Magic U14 Girls Black,237a9d9a-52c5-4136-b179-c01b10ad93ae,Saturday U14 Girls Division 4,https://www.playhq.com/basketball-victoria/org...,14
8,623c00bb-302b-4492-a988-90aacc3e7768,Magic U14 Boys Purple,d900616b-0f52-4d3e-8090-4eaf877491bb,Saturday U14 Boys Division 5,https://www.playhq.com/basketball-victoria/org...,14
9,a1e571f6-8ac4-49d2-9787-f62b35947eca,Magic U14 Boys Black,b877a73b-037d-460b-acb5-a2c45e5debee,Saturday U14 Boys Division 2,https://www.playhq.com/basketball-victoria/org...,14


Next get all upcoming games for the Club's teams.

In [5]:
upcoming_games_df = phq_club.get_games(teams_df, pd.to_datetime(GAME_DATE).tz_localize(TIMEZONE))
upcoming_games_df[phq.GAMES_COLS]


2022-08-04 12:19:12 DEBUG Extracting games for team: ('d7486008-b2fe-47db-8011-01265eaf1cfe', 'Magic U8 Mixed Purple')
2022-08-04 12:19:12 INFO Games extracted for team: Magic U8 Mixed Purple
2022-08-04 12:19:12 DEBUG Extracting games for team: ('8d635867-0718-4f70-a1b8-1a224992d294', 'Magic U16 Girls Gold')
2022-08-04 12:19:13 INFO Games extracted for team: Magic U16 Girls Gold
2022-08-04 12:19:13 DEBUG Extracting games for team: ('69d69214-641d-4ece-a2ce-ca9534865553', 'Magic U16 Boys Purple')
2022-08-04 12:19:13 INFO Games extracted for team: Magic U16 Boys Purple
2022-08-04 12:19:13 DEBUG Extracting games for team: ('135c3bab-3004-4426-b9c6-3669b48ce271', 'Magic U16 Boys Gold')
2022-08-04 12:19:13 INFO Games extracted for team: Magic U16 Boys Gold
2022-08-04 12:19:13 DEBUG Extracting games for team: ('99f369ed-5c63-40e7-8591-8cd18fba7ec0', 'Magic U14 Boys Gold')
2022-08-04 12:19:13 INFO Games extracted for team: Magic U14 Boys Gold
2022-08-04 12:19:13 DEBUG Extracting games for tea

,id,team_name,team_id,status,url,createdAt,updatedAt,pool,competitors,grade_id,...,venue_surfaceAbbreviation,venue_address_line1,venue_address_postcode,venue_address_suburb,venue_address_state,venue_address_country,venue_address_latitude,venue_address_longitude,schedule_timestamp,venue
0,68d5adf0-fec6-42ed-a529-97b474c70930,U8 Mixed Purple,d7486008-b2fe-47db-8011-01265eaf1cfe,UPCOMING,https://www.playhq.com/basketball-victoria/org...,2022-05-17 09:04:56+10:00,2022-05-17 09:04:56+10:00,None,[{'id': 'd7486008-b2fe-47db-8011-01265eaf1cfe'...,be7076cb-a1de-4033-8082-b52c8c149861,...,Crt1,25 Outlook Road,3058,Coburg North,VIC,Australia,-37.73315,144.97684,2022-08-06 08:30:00+10:00,NaN
1,77ddcb16-0df2-4f59-87f8-cc1316226017,U16 Girls Gold,8d635867-0718-4f70-a1b8-1a224992d294,UPCOMING,https://www.playhq.com/basketball-victoria/org...,2022-05-16 22:01:16+10:00,2022-08-02 11:38:24+10:00,None,[{'id': '8d635867-0718-4f70-a1b8-1a224992d294'...,19b86372-2ad5-4a5a-9592-467f826f7d63,...,Crt4,25 Outlook Road,3058,Coburg North,VIC,Australia,-37.73315,144.97684,2022-08-06 13:00:00+10:00,NaN
2,70557a7e-8193-4115-a17c-eaec757bc143,U16 Boys Purple,69d69214-641d-4ece-a2ce-ca9534865553,UPCOMING,https://www.playhq.com/basketball-victoria/org...,2022-05-31 11:21:21+10:00,2022-05-31 11:21:21+10:00,None,[{'id': '69d69214-641d-4ece-a2ce-ca9534865553'...,be4b2b9c-0bc1-44e2-ac3a-b89bb10230f8,...,Crt2,26-36 King Street,3047,Dallas,VIC,Australia,-37.67041,144.94180,2022-08-06 13:45:00+10:00,NaN
3,2ed999cd-2ee5-4783-ba5d-735ea52c8a11,U16 Boys Gold,135c3bab-3004-4426-b9c6-3669b48ce271,UPCOMING,https://www.playhq.com/basketball-victoria/org...,2022-05-31 11:46:42+10:00,2022-05-31 11:46:42+10:00,None,[{'id': '135c3bab-3004-4426-b9c6-3669b48ce271'...,ceac1ec4-c86b-47c6-9e67-124d923ea713,...,Crt1,26-36 King Street,3047,Dallas,VIC,Australia,-37.67041,144.94180,2022-08-06 13:00:00+10:00,NaN
4,8ac0b435-a8b4-420d-90c1-aa3ad2b72989,U14 Boys Gold,99f369ed-5c63-40e7-8591-8cd18fba7ec0,UPCOMING,https://www.playhq.com/basketball-victoria/org...,2022-05-31 11:20:16+10:00,2022-08-03 11:37:53+10:00,None,[{'id': '99f369ed-5c63-40e7-8591-8cd18fba7ec0'...,d900616b-0f52-4d3e-8090-4eaf877491bb,...,Crt1,19-25 St Georges Road,3070,Northcote,VIC,Australia,-37.77420,144.98985,2022-08-06 10:45:00+10:00,NaN
5,b6d8303d-d57f-4e6e-b915-7390cf56d562,U14 Girls Purple,8b2bc68b-9fda-435c-9c13-297d1d3187d1,UPCOMING,https://www.playhq.com/basketball-victoria/org...,2022-06-13 10:06:55+10:00,2022-06-13 10:06:55+10:00,None,[{'id': '8b2bc68b-9fda-435c-9c13-297d1d3187d1'...,b0785b1f-bec6-448b-a76d-83e1a6371fb3,...,Crt2,26-36 King Street,3047,Dallas,VIC,Australia,-37.67041,144.94180,2022-08-06 13:00:00+10:00,NaN
6,b6d8303d-d57f-4e6e-b915-7390cf56d562,U14 Girls Gold,9ebeb141-4023-4744-84f1-f58957b966b2,UPCOMING,https://www.playhq.com/basketball-victoria/org...,2022-06-13 10:06:55+10:00,2022-06-13 10:06:55+10:00,None,[{'id': '8b2bc68b-9fda-435c-9c13-297d1d3187d1'...,b0785b1f-bec6-448b-a76d-83e1a6371fb3,...,Crt2,26-36 King Street,3047,Dallas,VIC,Australia,-37.67041,144.94180,2022-08-06 13:00:00+10:00,NaN
7,82cc1a37-7179-4b5e-ab0f-8d7c09930b7d,U14 Girls Black,2b844e90-6575-47bc-bb49-f55840f8e6f8,UPCOMING,https://www.playhq.com/basketball-victoria/org...,2022-06-12 22:19:23+10:00,2022-06-12 22:19:23+10:00,None,[{'id': '2b844e90-6575-47bc-bb49-f55840f8e6f8'...,237a9d9a-52c5-4136-b179-c01b10ad93ae,...,Crt2,26-36 King Street,3047,Dallas,VIC,Australia,-37.67041,144.94180,2022-08-06 11:30:00+10:00,NaN
8,0d175857-4f84-4f53-8888-29caebd66d4b,U14 Boys Purple,623c00bb-302b-4492-a988-90aacc3e7768,UPCOMING,https://www.playhq.com/basketball-victoria/org...,2022-05-31 11:20:16+10:00,2022-08-02 09:46:49+10:00,None,[{'id': '623c00bb-302b-4492-a988-90aacc3e7768'...,d900616b-0f52-4d3e-8090-4eaf877491bb,...,Crt1,Cornwall Road,3044,Pascoe Vale,VIC,Australia,-37.71901,144.93687,2022-08-06 11:30:00+10:00,NaN
9,2bf4fff5-c765-4f34-a0e4-ecb45f2d79bf,U14 Boys Black,a1e571f6-8ac4-49d2-9787-f62b35947eca,UPCOMING,https://www.playhq.c

In [ ]:
game_rounds = upcoming_games_df['round.name'].unique()
game_date = upcoming_games_df['schedule.date'].unique()
if len(game_date) == 1:
    game_date = pd.to_datetime(game_date[0])
else:
    print("There are more than one date!: ", game_date)
    exit(1)

print(f'Games extracted for rounds {game_rounds} on date: {game_date.strftime("%a %Y/%m/%d")}')

## 3. Convert to Teams App CSV format

Next, we conver tthe PlayHQ upcoming games to Teams App format so we can produce a CSV file to be imported into Teams App.

In [ ]:
games_tapps_df = phq.to_teamsapp_schedule(upcoming_games_df, desc_template=DESC_TAPP)
print("Done computing the games for Teams App")
games_tapps_df

## 4. Prepare dataframe for Teams App CSV

First filter and re-order columns relevant to Teams App schedule format, and then produce corresponding CSV.

In [ ]:
TAPP_COLS_CSV = ['event_name', 'team_name', 'start_date', 'end_date', 'start_time', 'end_time', 'description', 'venue', 'location', 'access_groups', 'rsvp', 'comments', 'attendance_tracking', 'duty_roster', 'ticketing']

team_apps_csv_df = games_tapps_df.loc[:, TAPP_COLS_CSV]
team_apps_csv_df.sample(4)


In [ ]:
# Inspect description of one record
print(team_apps_csv_df.iloc[4]['event_name'])
print(team_apps_csv_df.iloc[4]['description'])


### Extract and add BYE games

In [ ]:
# Extract the date of the round
date = team_apps_csv_df.iloc[1]['start_date']

print(f"Extract BYE games for games on {date.strftime('%a %d/%m/%Y')}")

# Extract teams that do not have a game
playing_teams = games_tapps_df['team_id'].tolist()
bye_teams_df = teams_df.loc[~teams_df['id'].isin(playing_teams)].copy()

# Add entries for BYE teams
if not bye_teams_df.empty:  # there are BYE games
    print("There are BYE games...")

    bye_teams_df = bye_teams_df[['id', 'name']]

    bye_teams_df['team_name'] = bye_teams_df.apply(lambda x: re.search("U.*", x['name']).group(0), axis=1)
    bye_teams_df['access_groups'] = bye_teams_df['team_name']
    bye_teams_df['event_name'] = bye_teams_df['team_name'] + " - BYE"
    bye_teams_df['start_date'] = date
    bye_teams_df['end_date'] = date
    bye_teams_df['start_time'] = datetime.time(hour=0,minute=0,second=0)
    bye_teams_df['end_time'] = datetime.time(hour=0,minute=0,second=0)
    bye_teams_df['description'] = DESC_BYE_TAPP
    bye_teams_df['location'] = ""
    bye_teams_df['venue'] = "BYE"

    bye_teams_df['rsvp'] = 0
    bye_teams_df['comments'] = 0
    bye_teams_df['attendance_tracking'] = 0
    bye_teams_df['duty_roster'] = 0
    bye_teams_df['ticketing'] = 0
    bye_teams_df['reference_id'] = ""

    bye_teams_df = bye_teams_df[TAPP_COLS_CSV]
    team_apps_csv_df = pd.concat([team_apps_csv_df, bye_teams_df])
    team_apps_csv_df.drop_duplicates(inplace=True)

bye_teams_df

Let's report the teams that have a BYE

In [ ]:
if bye_teams_df.size > 0:
    for x in bye_teams_df['team_name'].values:
        print(x)
else:
    print("No BYE games this round...")

In [ ]:
team_apps_csv_df.columns
team_apps_csv_df[['team_name', 'start_time', 'venue']]

## 5. Save to CSV file for Teams App import

Finally, we save the data to a CSV file that can be imported into the [SCHEDULE of TeamsApp for all Entries](https://brunswickmagicbasketball.teamapp.com/clubs/263995/events?_list=v1&team_id=all).

In [ ]:
# date_games = team_apps_csv_df.iloc[1]['start_date']

file_csv = os.path.join(PATH_SAVE_CSV, f"schedule-teamsapp-{date.strftime('%Y_%m_%d')}.csv")

print(f'Saving schedule for round {game_rounds} on date: {game_date.strftime("%a %Y/%m/%d")}')
print('File to save TeamApp schedule:', file_csv)
team_apps_csv_df.to_csv(file_csv, index=False)

# Get one team game

In [10]:
team = "Magic U14 Girls Gold"

team_id = teams_df.query('name == @team')['id'].values[0]

game_team_df = phq_club.get_team_fixture_df(team_id)
# game_team_df.columns


# game_team_df.iloc[0]['schedule.timestamp']


game_date = pd.to_datetime(GAME_DATE).tz_localize(TIMEZONE) + pd.Timedelta(days=-20)
# game_date
game_team_df.query('schedule_timestamp >= @game_date')
# game_team_df.columns
# game_team_df.iloc[0]['schedule.date']

# upcoming_games_df = phq_club.get_games_by_status(teams_df)
# upcoming_games_df[]

,id,status,url,createdAt,updatedAt,pool,competitors,grade_id,grade_name,grade_url,...,venue_surfaceName,venue_surfaceAbbreviation,venue_address_line1,venue_address_postcode,venue_address_suburb,venue_address_state,venue_address_country,venue_address_latitude,venue_address_longitude,schedule_timestamp
9,d30bb11f-df5e-4b6a-b94f-64a7eab58450,FINAL,https://www.playhq.com/basketball-victoria/org...,2022-06-13 10:06:54+10:00,2022-07-25 20:10:29+10:00,None,[{'id': '35c394f3-8127-48c1-92d7-0d3ea39f8c4e'...,b0785b1f-bec6-448b-a76d-83e1a6371fb3,Saturday U14 Girls Division 3,https://www.playhq.com/basketball-victoria/org...,...,Court 1,Crt1,9 Hillcrest Road,3046,Oak Park,VIC,Australia,-37.713213,144.925617,2022-07-23 13:00:00+10:00
10,133b942b-0a3e-438a-b2aa-969d11871c9d,FINAL,https://www.playhq.com/basketball-victoria/org...,2022-06-13 10:06:55+10:00,2022-08-03 10:02:41+10:00,None,[{'id': 'c7aaa420-270a-47b0-8a05-bed7e2c98499'...,b0785b1f-bec6-448b-a76d-83e1a6371fb3,Saturday U14 Girls Division 3,https://www.playhq.com/basketball-victoria/org...,...,Court 1,Crt1,26-36 King Street,3047,Dallas,VIC,Australia,-37.670410,144.941800,2022-07-30 12:15:00+10:00
11,b6d8303d-d57f-4e6e-b915-7390cf56d562,UPCOMING,https://www.playhq.com/basketball-victoria/org...,2022-06-13 10:06:55+10:00,2022-06-13 10:06:55+10:00,None,[{'id': '8b2bc68b-9fda-435c-9c13-297d1d3187d1'...,b0785b1f-bec6-448b-a76d-83e1a6371fb3,Saturday U14 Girls Division 3,https://www.playhq.com/basketball-victoria/org...,...,Court 2,Crt2,26-36 King Street,3047,Dallas,VIC,Australia,-37.670410,144.941800,2022-08-06 13:00:00+10:00


# Testing area

In [ ]:
# short_url = shorten_url.short("https://www.playhq.com/basketball-victoria/org/coburg-basketball-association/junior-domestic-winter-2022/saturday-u-10-girls-division-2/game-centre/b8a62f18")
# short_url

# s = pyshorteners.Shortener()
# #  s.bitly.short('http://www.google.com')
# s.dagd.short('http://www.google.com')

# shorten_url('http://www.google.com')

In [ ]:
# id_team = teams_df.loc[teams_df['name'] == "Magic U12 Boys Black"]['id'].values[0]
# id_team

# data_json = phq_club.get_json(f'teams/{id_team}/fixture')
# games_team_df = pd.json_normalize(data_json['data'])

# games_team_df.columns
# upcoming_games = games_team_df.loc[games_team_df['status'] == "UPCOMING"]

# upcoming_games.empty
